In [13]:
#!/usr/bin/python

""" 
The first two cells are intended to build the database structure and to populate it from csv files.
In reality, the database would already exist and we would simply need to connect using the create_engine function.
"""

""" Building database structure """

#Importing standard libraries
import os
import sys
import datetime

from sqlalchemy import MetaData, Column, ForeignKey, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

#Instantiating base
Base = declarative_base()

#Creating tables
class Customer(Base):
    
    __tablename__ = 'customer'
    
    id = Column(Integer, primary_key = True)
    name = Column(String(80), nullable = False)
    
    #decorator
    @property
    def serialize(self):
        return {
            'id': self.id, 
            'name': self.name, 
            }

class Address(Base):
    
    __tablename__ = 'address'
    
    id = Column(Integer, primary_key = True)
    street = Column(String(80), nullable = False)
    zip = Column(String(5), nullable = False)
    country = Column(String(80), nullable = False)
    
class Product(Base):
    
    __tablename__ = 'product'
    
    id = Column(Integer, primary_key = True)
    name = Column(String(80), nullable = False)
    description = Column(String(250))
    
    #decorator
    @property
    def serialize(self):
        return {
            'id': self.id, 
            'name': self.name, 
            'description': self.description, 
            }
    
class Sale(Base):
    
    __tablename__ = 'sale'
    
    id = Column(Integer, primary_key = True)
    description = Column(String(250), nullable = False)
    price = Column(String(8), nullable = False)
    date = Column(String(10), nullable = False)
    
    
    customer_id = Column(Integer, ForeignKey('customer.id'))
    customer = relationship(Customer)
    
    product_id = Column(Integer, ForeignKey('product.id'))
    product = relationship(Product)

#Instantiating engine
engine = create_engine('sqlite:///database.db')
Base.metadata.create_all(engine)

#Initializing session
DBSession = sessionmaker(bind=engine)
session = DBSession()

#Finishing
print 'database created'

database created


In [14]:
#!/usr/bin/python

""" Populating database from csv files """

#Importing libraries
from numpy import genfromtxt

#Clearing database
Base.metadata.drop_all(bind=engine)
#Reinitializing database
Base.metadata.create_all(bind=engine)

#Creating file importer
def Load_Data(file_name, lines_skip=0):
    data = genfromtxt(file_name, dtype=None, delimiter=',', skip_header=lines_skip)
    return data.tolist()

#Getting list of tables from text file
tables = Load_Data("tables.txt")
for i in range(len(tables)):
    #Getting table name
    table_name = tables[i]
    
    #Getting data from source file
    data = Load_Data(table_name + ".csv", 1)
    
    try:
        table = locals()[table_name]

        #Getting table fields
        table_keys = table.__table__.columns.keys()

        #Browsing data from source file
        for j in range(len(data)):
            #Initializing dictionary for new record
            dd = {}

            #Browsing table fields and updating dictionary
            for k in range(len(table_keys)):
                dd.update({table_keys[k]:data[j][k]})

            #Adding record
            #print 'dd = ', dd
            record = table(**dd)
            session.add(record)

        #Saving changes
        session.commit()

        #Printing records for check
        print 'records in ', table.__name__, ' table:'
        items = session.query(table).all()
        for item in items:
            print item
            
    except:
        print table_name, ' not found in ', engine, ' database'
        
session.close()

records in  Customer  table:
records in  Product  table:
records in  Sale  table:
